In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp shared

# Shared Functions

> This module contains various shared helper functions for examples.

In [ ]:
#hide
#exporti
import numpy as np
import pandas as pd
import scipy as sc
import pickle
from scipy.special import digamma
from pandas.tseries.holiday import AbstractHolidayCalendar, USMartinLutherKingJr, USMemorialDay, Holiday, USLaborDay, \
    USThanksgivingDay
import os
import pickle
import zlib

In [ ]:
#exporti

def load_interpolators():
    
    pkg_data_dir = os.path.dirname(os.path.abspath(__file__)) + '/pkg_data'
    #pkg_data_dir = os.getcwd().split('pybats')[0] + 'pybats/pybats/pkg_data'
    #pkg_data_dir = globals()['_dh'][0] + '/pkg_data'

    try:
        with open(pkg_data_dir + '/interp_beta.pickle.gzip', 'rb') as fl:
            interp_beta = pickle.loads(zlib.decompress(fl.read()))

        with open(pkg_data_dir + '/interp_gamma.pickle.gzip', 'rb') as fl:
            interp_gamma = pickle.loads(zlib.decompress(fl.read()))

    except:
        print('WARNING: Unable to load interpolator. Code will run slower.')
        interp_beta, interp_gamma = None, None
        
    return interp_beta, interp_gamma

In [ ]:
#exporti
# I need this helper in a module file for pickle reasons ...
def transformer(ft, qt, fn1, fn2):
    return np.exp(np.ravel(fn1(ft, np.sqrt(qt), grid=False))), \
           np.exp(np.ravel(fn2(ft, np.sqrt(qt), grid=False)))

In [ ]:
#exporti
def gamma_transformer(ft, qt, fn):
    alpha = np.ravel(np.exp(fn(np.sqrt(qt))))
    beta = np.exp(digamma(alpha) - ft)
    return alpha, beta

In [ ]:
#exporti
def trigamma(x):
    return sc.special.polygamma(x=x, n=1)

In [ ]:
#exporti
def save(obj, filename):
    with open(filename, "wb") as file:
        pickle.dump(obj, file=file)

In [ ]:
#exporti
def load(filename):
    with open(filename, "rb") as file:
        tmp = pickle.load(file)
    return tmp

In [ ]:
#exporti
def define_holiday_regressors(X, dates, holidays=None):
    """
    Add columns to the predictor matrix X for a specified list of holidays

    :param X: (Array) Predictor matrix without columns for the holidays
    :param dates: Dates
    :param holidays: (List) holidays
    :return: Updated predictor matrix
    """
    if holidays is not None:
        if len(holidays) > 0:
            if X is None:
                n = len(dates)
            else:
                n = X.shape[0]

            for holiday in holidays:
                cal = AbstractHolidayCalendar()
                cal.rules = [holiday]
                x = np.zeros(n)
                x[dates.isin(cal.holidays())] = 1
                if X is None:
                    X = x
                else:
                    X = np.c_[X, x]

            return X
        else:
            return X
    else:
        return X

In [ ]:
#export
def load_standard_holidays():
    """
    Load in a list of standard holidays
    """
    
    holidays = [USMartinLutherKingJr,
                USMemorialDay,
                Holiday('July4', month=7, day=4),
                USLaborDay,
                # Holiday('Thanksgiving_1DB', month=11, day=1, offset=pd.DateOffset(weekday=WE(4))),
                USThanksgivingDay,
                # Holiday('Christmas_1DB', month=12, day=24),
                Holiday('Christmas', month=12, day=25),
                Holiday('New_Years_Eve', month=12, day=31),
                ]
    return holidays

In [ ]:
#export
def cov2corr(cov):
    """
    Transform a covariance matrix into a correlation matrix. Useful for understanding coefficient correlations
    """
    D = np.sqrt(cov.diagonal()).reshape(-1, 1)
    return cov / D / D.T

In [ ]:
#export
def load_sales_example():
    """
    Read data for the first sales forecasting example
    """
    data_dir = os.path.dirname(os.path.abspath(__file__)) + '/pkg_data/'
    return pd.read_csv(data_dir + 'sales.csv', index_col=0)[['Sales', 'Advertising']]

In [ ]:
#export
def load_sales_example2():
    """
    Read data for the second sales forecasting example
    """
    data_dir = os.path.dirname(os.path.abspath(__file__)) + '/pkg_data/'
    data = pd.read_pickle(data_dir + 'sim_sales_data')
    data = data.set_index('Date')
    return data

In [ ]:
#export
def load_dcmm_latent_factor_example():
    """
    Read data for the DCMM latent factor example
    """
    data_dir = os.path.dirname(os.path.abspath(__file__)) + '/pkg_data/'
    data = load(data_dir + 'dcmm_latent_factor_data')
    return data

In [ ]:
#export
def load_dbcm_latent_factor_example():
    """
    Read data for the DBCM latent factor example
    """
    data_dir = os.path.dirname(os.path.abspath(__file__)) + '/pkg_data/'
    data = load(data_dir + 'dbcm_latent_factor_data')
    return data

In [ ]:
#export
def load_us_inflation():
    """
    Read in quarterly US inflation data
    """
    data_dir = os.path.dirname(os.path.abspath(__file__)) + '/pkg_data/'
    data = pd.read_csv(data_dir + 'us_inflation.csv')
    return data

In [ ]:
#export
def load_us_inflation_forecasts():
    """
    Read in quarterly US inflation data along with forecasts from 4 models
    """
    data_dir = os.path.dirname(os.path.abspath(__file__)) + '/pkg_data/'
    
    data = pd.read_csv(data_dir + 'bps_inflation.csv')
    dates = data.values[:,0]
    agent_mean = pd.read_csv(data_dir + 'bps_agent_mean.csv')
    agent_mean.columns = ['Dates', '1', '2', '3', '4']
    agent_mean.set_index('Dates', inplace=True)
    
    agent_var = pd.read_csv(data_dir + 'bps_agent_var.csv').values
    agent_dof = pd.read_csv(data_dir + 'bps_agent_dof.csv').values
    agent_var[:,1:] = agent_var[:,1:] * agent_dof[:,1:] / (agent_dof[:,1:]-2) # Adjust the agent variance for d.o.f. b/c they're t-distributed
    agent_var = pd.DataFrame(agent_var)
    agent_var.columns = ['Dates', '1', '2', '3', '4']
    agent_var.set_index('Dates', inplace=True)
    
    dates = pd.date_range('1977-09-01', '2014-12-31', freq='3M')
    Y = data['Inflation'].values
    
    data = {'Inflation':Y, 'model_mean':agent_mean, 'model_var':agent_var, 'Dates':dates}
    
    return data

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_dglm.ipynb.
Converted 01_update.ipynb.
Converted 02_forecast.ipynb.
Converted 03_define_models.ipynb.
Converted 04_seasonal.ipynb.
Converted 05_analysis.ipynb.
Converted 06_conjugates.ipynb.
Converted 07_point_forecast.ipynb.
Converted 08_loss_functions.ipynb.
Converted 09_plot.ipynb.
Converted 10_shared.ipynb.
Converted 11_dcmm.ipynb.
Converted 12_dbcm.ipynb.
Converted 13_latent_factor.ipynb.
Converted 14_latent_factor_fxns.ipynb.
Converted index.ipynb.
